# Определение пар товаров, которые покупают вместе

Пары товаров подобраны по ассоциативным правилам (Association Rules) с помощью алгоритма apriori, реализованного в библиотеке MLxtend для Python (http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/)

### Подключение библиотек, подгрузка, группировка, выборка данных для модели

In [247]:
# подключаем библиотеки numpy, pandas, mlxtend

import pandas as pd
import numpy as np

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori as apriori
from mlxtend.frequent_patterns import association_rules

In [248]:
# подгружаем данныые, смотрим первые 5 строк
data = pd.read_csv('dataset_for_test_20200729_cleaned.csv',sep=';', decimal=",")
data.head()

,order_id,_date,user_id,plu_id,category_id,parent_category_id,product_quantity,product_sum
0,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,3383918,855.0,1105.0,1.0,49.9
1,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,3489545,853.0,849.0,1.0,89.9
2,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,2068955,839.0,834.0,1.0,99.9
3,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,3435295,838.0,834.0,2.0,33.8
4,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,3435297,838.0,834.0,2.0,31.8


In [249]:
# проверяем размерность таблицы
data.shape

(332642, 8)

In [250]:
# делаем выборку только двух полей для модели: id заказа, id товара; смотрим первые 5 строк
df = data.loc[:, ['order_id', 'plu_id']]
df.head()

,order_id,plu_id
0,5efc2687f59a7f82441fe4ca,3383918
1,5efc2687f59a7f82441fe4ca,3489545
2,5efc2687f59a7f82441fe4ca,2068955
3,5efc2687f59a7f82441fe4ca,3435295
4,5efc2687f59a7f82441fe4ca,3435297


In [251]:
# проверяем размерность данных выборки, на всякий случай
df.shape

(332642, 2)

In [252]:
# группируем плоские данные по заказам, продукты в одном заказе схлопываются в список в одной строке
df2 = df.groupby('order_id').agg(lambda x: x.tolist())
df2.head()

,plu_id
order_id,
5efc2687f59a7f82441fe4ca,"[3383918, 3489545, 2068955, 3435295, 3435297, ..."
5efc26a0d9c459f11a8e122f,"[1860, 2139858, 3170135, 3341462, 45463, 36955..."
5efc26a4d9c45903d38e1247,"[2137027, 1536, 3661004, 3664384, 3674166, 368..."
5efc26b4d9c459748e8e126c,"[3477505, 43693, 79108, 3639749, 3444750, 3682..."
5efc26c1f59a7f62941fe522,"[2144368, 18510, 3480546, 3224596, 3998557, 40..."


In [253]:
# смотрим размерность полученного агрегата
df2.shape

(23773, 1)

In [254]:
# датасет типа pandas data frame преобразовываем в обычный список для модели, которая на вход принимает объект типа список,
# и снова проверяем его длину
temp = df2['plu_id'].to_list()
len(temp)

23773

### Создание модели для поиска ассоциативных правил

In [255]:
# создаем объект модели, передаем в нее данные, обучаем
te = TransactionEncoder()
te_ary = te.fit(temp).transform(temp)
df = pd.DataFrame(te_ary, columns=te.columns_)

### В следующем участке кода определяются значения нескольких гиперпараметров для модели. Управляя ими, мы определяем результат ее работы. Условно, если задать высокие требования к модели,то на выходе получим мало пар (или совсем не получим). В данном экземпляре конкретные значения гиперпараметров найдены эмпирическим способом. 

------

Тут задаем уровень минимальной поддержки - фактически, это минимальная доля заказов в датасете, в которых есть пары товаров. 

In [256]:
# На датасете примера высокие значения этого параметра приводили к нулевому результату, возможно, из-за разношерстности заказов по
# товарным наборам. В итоге, чтобы модель заработала и дала выхлоп, остановимся на таком значении:
MIN_SUPPORT = 0.02

frequent_itemsets = apriori(df, min_support=MIN_SUPPORT, use_colnames=True)
# frequent_itemsets


#### Далее из полученного на пред.шаге набора правил отбираем пары с заданным уровнем качества. Пары фильтруются 2 раза, независимо друг от друга как альтернативные подходы.


---

Тут отбираем по Confidence. Он о том, как часто это правило работает для датасета (процент тех, кто покупает оба товара в проценте от тех, кто купил первый). 

In [257]:
# в данном случае эмипирически выбрано следующее минимальное значение: 
CONFIDENCE = 0.2

# запуск фильтрации
pairs_conf = association_rules(frequent_itemsets, metric="confidence", min_threshold=CONFIDENCE)

# вывод результата работы модели со статистическими показателями, выборка нескольких строк
pairs_conf.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(3759),(4072),0.055441,0.179027,0.021327,0.384674,2.148696,0.011401,1.334208
1,(5013),(4072),0.120052,0.179027,0.033357,0.277856,1.552035,0.011865,1.136855
2,(21292),(4072),0.065789,0.179027,0.026080,0.396419,2.214304,0.014302,1.360172
3,(45463),(4072),0.105203,0.179027,0.033820,0.321471,1.795663,0.014986,1.209932
4,(4072),(45505),0.179027,0.165398,0.057965,0.323778,1.957574,0.028354,1.234214


В списке выше идшки товаров, входящие в рекомендованные пары, приведены в полях antecedents и consequents.

In [258]:
# делаем выборку полей только с идшками пар товаров
pairs_conf_data = pairs_conf.loc[:, ['antecedents', 'consequents']]

# так как в таблице с результатами информация дублируется (каждая пара два раз, где элементы переставлены местами), то оставляем 
# только уникальные строки 
pairs_conf_no_duplicates = pairs_conf_data.loc[::2, ]
pairs_conf_no_duplicates

,antecedents,consequents
0,(3759),(4072)
2,(21292),(4072)
4,(4072),(45505)
6,(52112),(4072)
8,(79108),(4072)
10,(3638820),(4072)
12,(3639749),(4072)
14,(5013),(45505)
16,(5013),(79108)
18,(45505),(45463)


### **Итого: получены 17 пар товаров при заданных значениях гиперпараметров.**

------

Ниже второй подход, альтернатива первому (который по Confidence). Во втором подходе отбираем пары по показателю Lift. Он о достоверности правила (отношение: "оба товара покупаются вместе" / "товар покупался вообще"). Значение должно быть больше 1 для отбора работающих правил.

In [259]:
# фильтруем по минимальном значению 1.7
LIFT = 1.7

pairs_lift = association_rules(frequent_itemsets, metric="lift", min_threshold=LIFT)
pairs_lift_data = pairs_lift.loc[:, ['antecedents', 'consequents']]

# убираем дубликаты
pairs_lift_no_duplicates = pairs_lift_data.loc[::2, ]
pairs_lift_no_duplicates

,antecedents,consequents
0,(4072),(3759)
2,(4072),(21292)
4,(4072),(45463)
6,(4072),(45505)
8,(4072),(53484)
10,(4072),(3638820)
12,(4072),(3639749)
14,(4072),(3695580)
16,(79108),(5013)
18,(45505),(45463)


### **Итого: получены 16 пар товаров при заданных критериях (гиперпараметрах)**

In [260]:
# сохранение результата во внешний файл
pairs_lift_no_duplicates.to_csv('result.csv')

In [261]:
# сохранение результата во внешний файл с очищенными символами

# небольшая функция
def make_output(df, filename='pairs_ouput.csv'):
    result = []
    temp = df.to_numpy()
    for row in temp:       
        left = str(row[0]).split('{')
        res_left = int(left[1].split('}')[0])
        right = str(row[1]).split('{')
        res_right = int(right[1].split('}')[0])
        result.append([res_left, res_right])    
    temp = pd.DataFrame(result)
    temp.to_csv(filename, index=False, header=False)

In [262]:
make_output(pairs_lift_no_duplicates)

---------
---------

### Приложение / дополнительный комментарий:

Любопытно, что данный алгоритм может объединять в одной из пар несколько товаров. Ниже вариант, где получена выборка бОльшего размера за счет низкого значения фильтрующего MIN_SUPPORT. В его результатах встречаются такие "связки". 
(Вывод не очищен от дублей)

In [266]:
MIN_SUPPORT2 = 0.01
frequent_itemsets2 = apriori(df, min_support=MIN_SUPPORT2, use_colnames=True)

CONFIDENCE2 = 0.2
pairs_conf2 = association_rules(frequent_itemsets2, metric="confidence", min_threshold=CONFIDENCE2)
pairs_conf_data2 = pairs_conf2.loc[:, ['antecedents', 'consequents']]

LIFT2 = 1.7
pairs_lift2 = association_rules(frequent_itemsets, metric="lift", min_threshold=LIFT2)
pairs_lift_data2 = pairs_lift2.loc[:, ['antecedents', 'consequents']]

In [267]:
pairs_conf_data2

,antecedents,consequents
0,(1536),(4072)
1,(1536),(45505)
2,(1536),(3639749)
3,(2166),(4072)
4,(3757),(4072)
...,...,...
151,"(45505, 45463)",(3639749)
152,"(3639749, 45463)",(45505)
153,"(45505, 53484)",(3638820)
154,"(45505, 3638820)",(53484)


In [268]:
pairs_lift_data2

,antecedents,consequents
0,(4072),(3759)
1,(3759),(4072)
2,(4072),(21292)
3,(21292),(4072)
4,(4072),(45463)
5,(45463),(4072)
6,(4072),(45505)
7,(45505),(4072)
8,(4072),(53484)
9,(53484),(4072)


---------

### Вывод: 
отбор пар может производиться с помощью алгоритма apriori на основе подхода определения ассоциативных правил. Значения конкретных гиперпараметров, влияющих на выход модели, целесообразно подбирать на практике, исходя из характеристик датасета (объем, разнообразие информации), требуемого уровня точности и полноты результата. 

-----